In [1]:
!pip install pycep-correios

     |████████████████████████████████| 100 kB 1.7 MB/s ta 0:00:011
     |████████████████████████████████| 41 kB 259 kB/s  eta 0:00:01
     |████████████████████████████████| 6.4 MB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 936 kB/s eta 0:00:011
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.1
    Uninstalling lxml-4.2.1:
      Successfully uninstalled lxml-4.2.1


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

hive_context = HiveContext(sc)

spark = SparkSession \
    .builder \
    .appName("Teste API") \
    .enableHiveSupport() \
    .getOrCreate()

geo = spark.read.orc('/datalake/dadosbrutos/olist_geolocation_dataset.orc')

+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
+---------------------------+---------------+---------------+----------------+-----------------+



In [116]:
from pycep_correios import get_address_from_cep, WebService, exceptions
import zeep
import pandas as pd
# All rows generated with the adress will saved into empty result variable list 
cep_array = [str(row.geolocation_zip_code_prefix) for row in geo.select('geolocation_zip_code_prefix').distinct().collect()]
result = []
sufixos = ['000', '010', '900', '970', '990', '999']

def getvalues(limit=None):
    counter = 0
    for i in cep_array:
        for sufixo in sufixos:
            try:
                cep = i + sufixo
                print(f"{counter}º coleta, Cep {cep}")
                endereco = get_address_from_cep(cep, webservice=WebService.CORREIOS)
                if(endereco["cep"] == ""): continue
                else: break
            except exceptions.CEPNotFound as e:
                continue
            except exceptions.InvalidCEP as e:
                continue
            except exceptions.BaseException as e:
                continue
        result.append(endereco) # Appending the result generated into empty result variable list
        counter += 1
        if counter == limit:break
    return result

if __name__ == "__main__":
    test = getvalues(limit=100)

# Creating a dataframe from function return result
df = pd.DataFrame(result)
df = df.drop_duplicates()
df.head(100)


0º coleta, Cep 13282000
1º coleta, Cep 13610000
1º coleta, Cep 13610010
2º coleta, Cep 14887000
3º coleta, Cep 16250000
4º coleta, Cep 17506000
5º coleta, Cep 18130000
5º coleta, Cep 18130010
6º coleta, Cep 18306000
7º coleta, Cep 76976000
8º coleta, Cep 77303000
9º coleta, Cep 77814000
9º coleta, Cep 77814010
10º coleta, Cep 77930000
11º coleta, Cep 78888000
12º coleta, Cep 79785000
13º coleta, Cep 80820000
14º coleta, Cep 83450000
15º coleta, Cep 84272000
16º coleta, Cep 85022000
17º coleta, Cep 86804000
18º coleta, Cep 86900000
19º coleta, Cep 87103000
19º coleta, Cep 87103010
19º coleta, Cep 87103900
19º coleta, Cep 87103970
20º coleta, Cep 87117000
20º coleta, Cep 87117010
20º coleta, Cep 87117900
20º coleta, Cep 87117970
20º coleta, Cep 87117990
20º coleta, Cep 87117999
21º coleta, Cep 88140000
22º coleta, Cep 88138000
22º coleta, Cep 88138010
22º coleta, Cep 88138900
22º coleta, Cep 88138970
22º coleta, Cep 88138990
22º coleta, Cep 88138999
23º coleta, Cep 88311000
24º coleta, C

,bairro,cep,cidade,logradouro,uf,complemento
0,Observatório,13282000,Vinhedo,Rua do Observatório,SP,(Villa Monteverde)
1,Centro,13610010,Leme,Avenida Benedito Landgraf,SP,None
2,Nova Jaboticabal,14887000,Jaboticabal,Avenida Carlos Berchieri,SP,- de 301 a 899 - lado ímpar
3,None,16250000,Clementina,None,SP,None
4,Somenzari,17506000,Marília,Avenida Castro Alves,SP,- até 1260/1261
...,...,...,...,...,...,...
82,Pico (Santo Aleixo),25912000,Magé,Rua Malvino Ferreira de Andrade,RJ,None
83,Cacuia,26082010,Nova Iguaçu,Travessa Tinguá,RJ,None
84,Centro,26112000,Belford Roxo,Rua Antônia,RJ,None
85,Eucaliptal,27264000,Volta Redonda,Rua Castro Alves,RJ,None


In [121]:
!pip install geopy

     |████████████████████████████████| 118 kB 7.3 MB/s eta 0:00:01


In [ ]:
coord_array = [[str(row.geolocation_lat), str(row.geolocation_lng)] 
             for row in geo.select('geolocation_lat', 'geolocation_lng').distinct().collect()]

In [34]:
from geopy.geocoders import Nominatim
import pandas as pd

def extrai_cep(coord):
    print(f"{counter}º chamada, coordenadas: {coord}", end ="\r")
    
    geolocator = Nominatim(user_agent="test_app")
    location = geolocator.reverse(coord)
    
    return({"cep":location.raw['address']['postcode'], 
            "lat":coord[0], 
            "lng":coord[1]})
    
location_list = []
limit = 25
counter = 0

for item in coord_array:
    location_list.append(extrai_lat_long(item))
    counter += 1
    if counter == limit: break
        
df = pd.DataFrame(location_list)
df = df.drop_duplicates()
df.head(100)

{'place_id': 181203399, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 323836768, 'lat': '-23.086153683546407', 'lon': '-45.689929168246415', 'display_name': 'Vila Santos, Caçapava, Região Imediata de São José dos Campos, Região Metropolitana do Vale do Paraíba e Litoral Norte, Região Geográfica Intermediária de São José dos Campos, São Paulo, Região Sudeste, 12280-015, Brasil', 'address': {'suburb': 'Vila Santos', 'city_district': 'Caçapava', 'town': 'Caçapava', 'municipality': 'Região Imediata de São José dos Campos', 'county': 'Região Metropolitana do Vale do Paraíba e Litoral Norte', 'state_district': 'Região Geográfica Intermediária de São José dos Campos', 'state': 'São Paulo', 'region': 'Região Sudeste', 'postcode': '12280-015', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-23.0863776', '-23.0851342', '-45.6910232', '-45.6896856']}
{'place_id': 213436411, 'licence': 'Data © OpenStreetMap contribut

{'place_id': 156965361, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 226505630, 'lat': '-23.297440382881508', 'lon': '-45.94457149212225', 'display_name': 'Rua Casemiro de Abreu, Parque dos Principes, Vila Zezé, Jacareí, Região Imediata de São José dos Campos, Região Metropolitana do Vale do Paraíba e Litoral Norte, Região Geográfica Intermediária de São José dos Campos, São Paulo, Região Sudeste, 12310600, Brasil', 'address': {'road': 'Rua Casemiro de Abreu', 'quarter': 'Parque dos Principes', 'suburb': 'Vila Zezé', 'city_district': 'Jacareí', 'city': 'Jacareí', 'municipality': 'Região Imediata de São José dos Campos', 'county': 'Região Metropolitana do Vale do Paraíba e Litoral Norte', 'state_district': 'Região Geográfica Intermediária de São José dos Campos', 'state': 'São Paulo', 'region': 'Região Sudeste', 'postcode': '12310600', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-23.2977551', '-23.2971

{'place_id': 151401906, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 208923265, 'lat': '-22.957094472182774', 'lon': '-45.47192729244024', 'display_name': 'Rua Codorna, Condomínio Residencial Colonial Village, Anhanguera, Pindamonhangaba, Região Imediata de Taubaté-Pindamonhangaba, Região Metropolitana do Vale do Paraíba e Litoral Norte, Região Geográfica Intermediária de São José dos Campos, São Paulo, Região Sudeste, 12421-681, Brasil', 'address': {'road': 'Rua Codorna', 'residential': 'Condomínio Residencial Colonial Village', 'suburb': 'Anhanguera', 'city_district': 'Pindamonhangaba', 'city': 'Pindamonhangaba', 'municipality': 'Região Imediata de Taubaté-Pindamonhangaba', 'county': 'Região Metropolitana do Vale do Paraíba e Litoral Norte', 'state_district': 'Região Geográfica Intermediária de São José dos Campos', 'state': 'São Paulo', 'region': 'Região Sudeste', 'postcode': '12421-681', 'country': 'Brasil', 'coun

KeyError: 'city_district'

In [35]:
from geopy.geocoders import Nominatim
import pandas as pd

def extrai_cep(coord):
    print(f"{counter}º chamada, coordenadas: {coord}", end ="\r")
    
    geolocator = Nominatim(user_agent="test_app")
    location = geolocator.reverse(coord)
    
    return({"cep":location.raw['address']['postcode'], 
            "lat":coord[0], 
            "lng":coord[1]})
    
location_list = []
limit = 25
counter = 0

for item in coord_array:
    location_list.append(extrai_lat_long(item))
    counter += 1
    if counter == limit: break
        
df = pd.DataFrame(location_list)
df = df.drop_duplicates()
df.head(100)

{'place_id': 181203399, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 323836768, 'lat': '-23.086153683546407', 'lon': '-45.689929168246415', 'display_name': 'Vila Santos, Caçapava, Região Imediata de São José dos Campos, Região Metropolitana do Vale do Paraíba e Litoral Norte, Região Geográfica Intermediária de São José dos Campos, São Paulo, Região Sudeste, 12280-015, Brasil', 'address': {'suburb': 'Vila Santos', 'city_district': 'Caçapava', 'town': 'Caçapava', 'municipality': 'Região Imediata de São José dos Campos', 'county': 'Região Metropolitana do Vale do Paraíba e Litoral Norte', 'state_district': 'Região Geográfica Intermediária de São José dos Campos', 'state': 'São Paulo', 'region': 'Região Sudeste', 'postcode': '12280-015', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-23.0863776', '-23.0851342', '-45.6910232', '-45.6896856']}
{'place_id': 213436411, 'licence': 'Data © OpenStreetMap contribut

{'place_id': 156965361, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 226505630, 'lat': '-23.297440382881508', 'lon': '-45.94457149212225', 'display_name': 'Rua Casemiro de Abreu, Parque dos Principes, Vila Zezé, Jacareí, Região Imediata de São José dos Campos, Região Metropolitana do Vale do Paraíba e Litoral Norte, Região Geográfica Intermediária de São José dos Campos, São Paulo, Região Sudeste, 12310600, Brasil', 'address': {'road': 'Rua Casemiro de Abreu', 'quarter': 'Parque dos Principes', 'suburb': 'Vila Zezé', 'city_district': 'Jacareí', 'city': 'Jacareí', 'municipality': 'Região Imediata de São José dos Campos', 'county': 'Região Metropolitana do Vale do Paraíba e Litoral Norte', 'state_district': 'Região Geográfica Intermediária de São José dos Campos', 'state': 'São Paulo', 'region': 'Região Sudeste', 'postcode': '12310600', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-23.2977551', '-23.2971

KeyboardInterrupt: 

In [112]:
df['cep'].str.contains('49379', na=False).sum()

TypeError: 'Column' object is not callable

In [118]:
geo.where('geolocation_zip_code_prefix = 3959').show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       3959|-23.592625628798753|-46.486169266076935|       sao paulo|               SP|
|                       3959| -23.60090574271452| -46.48096445337296|       sao paulo|               SP|
|                       3959| -23.59482766250217| -46.48469168443072|       sao paulo|               SP|
|                       3959|-23.595497999999992|-46.485124569940545|       sao paulo|               SP|
|                       3959| -23.59490731840087| -46.48462337893042|       sao paulo|               SP|
|                       3959|-23.596771306134208| -46.48600562683938|       sao paulo|               SP|
|                       3959|-23.598978684430712|-46.48

In [89]:
geo.where('geolocation_zip_code_prefix = 83450').show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                      83450|-25.214393219361128| -49.11148701241143| bocaiuva do sul|               PR|
|                      83450|  -25.2118243700817|-49.111183147599384| bocaiuva do sul|               PR|
|                      83450|-25.209980898769608| -49.10826946067007| bocaiuva do sul|               PR|
|                      83450|-25.212055442213092| -49.11459305409779| bocaiúva do sul|               PR|
|                      83450| -25.21650748262607| -49.10552043329379| bocaiuva do sul|               PR|
|                      83450|-25.212696366203364| -49.11534552733716| bocaiuva do sul|               PR|
|                      83450| -25.20968452337417|-49.11